In [10]:

import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')


In [11]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


In [13]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

Exemplo de texto original:
Este filme é incrível, adorei a atuação do protagonista

Depois do pré-processamento:
['filme', 'incrível', 'adorei', 'atuação', 'protagonista']


In [14]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)




In [18]:
# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")

Modelo treinado com 44 palavras no vocabulário


In [16]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras




Algumas palavras do vocabulário:
['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo']


In [17]:
# Verificar o vetor de uma palavra específica
if 'filme' in model.wv:
    vetor_filme = model.wv['filme']
    print(f"\nVetor da palavra 'filme' (primeiras 10 dimensões):")
    print(vetor_filme[:10])
    print(f"Dimensionalidade do vetor: {len(vetor_filme)}")


Vetor da palavra 'filme' (primeiras 10 dimensões):
[-0.00053898  0.00023755  0.00510847  0.00900957 -0.00930602 -0.00711791
  0.00645901  0.00897323 -0.00500892 -0.00376375]
Dimensionalidade do vetor: 100


In [20]:
len(model.wv.key_to_index)

44

In [21]:
if 'filme' in model.wv:
    vector_filme = model.wv['filme']
    print(vector_filme)

[-5.3898495e-04  2.3754722e-04  5.1084701e-03  9.0095662e-03
 -9.3060182e-03 -7.1179080e-03  6.4590140e-03  8.9732325e-03
 -5.0089178e-03 -3.7637462e-03  7.3839580e-03 -1.5429476e-03
 -4.5339474e-03  6.5549053e-03 -4.8634596e-03 -1.8150365e-03
  2.8793407e-03  9.9878211e-04 -8.2852831e-03 -9.4514769e-03
  7.3103900e-03  5.0675846e-03  6.7592384e-03  7.6005177e-04
  6.3530109e-03 -3.4019963e-03 -9.5264451e-04  5.7722717e-03
 -7.5249239e-03 -3.9323601e-03 -7.5148693e-03 -9.2767383e-04
  9.5453048e-03 -7.3271766e-03 -2.3372965e-03 -1.9302175e-03
  8.0870856e-03 -5.9312731e-03  4.2869444e-05 -4.7512343e-03
 -9.6031642e-03  5.0065401e-03 -8.7594036e-03 -4.3953191e-03
 -3.3294396e-05 -2.9962539e-04 -7.6586660e-03  9.6185729e-03
  4.9843234e-03  9.2339637e-03 -8.1531862e-03  4.4953576e-03
 -4.1356767e-03  8.2382484e-04  8.4956065e-03 -4.4643418e-03
  4.5180386e-03 -6.7829816e-03 -3.5452251e-03  9.4016502e-03
 -1.5767787e-03  3.2035878e-04 -4.1417871e-03 -7.6862508e-03
 -1.5016500e-03  2.46714

In [22]:
# Encontrar palavras mais similares a 'filme'
if 'filme' in model.wv:
    similares = model.wv.most_similar('filme', topn=5)
    print("\nPalavras mais similares a 'filme':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")



Palavras mais similares a 'filme':
direção: 0.2189
assistindo: 0.2168
impressionantes: 0.1955
cinema: 0.1694
efeitos: 0.1520


In [23]:
print([nome for nome in list(api.info()['models'].keys())[:10]])

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']


In [24]:
modelo_pretreinado = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [26]:
# Salvar o modelo
model.save("word2vec_filmes.model")

# Carregando o modelo salvo
modelo_carregado = Word2Vec.load("word2vec_filmes.model")
print("Modelo carregado com sucesso!")


Modelo carregado com sucesso!


In [27]:
# Listar modelos disponíveis
print("Alguns modelos disponíveis no gensim-data:")
print([nome for nome in list(api.info()['models'].keys())[:10]])

# Carregar um modelo pré-treinado (word2vec-google-news-300)
# Nota: Isso pode demorar um pouco na primeira execução (download)
try:
    # Para português, você pode tentar 'word2vec-portuguese' se disponível
    # ou usar modelos do NILC: http://nilc.icmc.usp.br/embeddings
    modelo_pretreino = api.load("glove-wiki-gigaword-100")  # Mais rápido que word2vec-google-news-300
    print(f"Modelo pré-treinado carregado com {len(modelo_pretreino.key_to_index)} palavras")

    # Verificar palavras similares usando modelo pré-treinado
    if 'computer' in modelo_pretreino:
        similares = modelo_pretreino.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")
except Exception as e:
    print(f"Erro ao carregar modelo pré-treinado: {e}")
    print("Tente outro modelo ou prossiga sem esta parte")


Alguns modelos disponíveis no gensim-data:
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']
Modelo pré-treinado carregado com 400000 palavras

Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [28]:
# Função para calcular similaridade entre pares de palavras
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados

# Pares de palavras para testar
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]

# Calcular similaridades
similaridades = calcular_similaridade(model, pares)

# Exibir resultados
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")



Similaridade entre pares de palavras:
filme - cinema: 0.1693534404039383
bom - ruim: Uma ou ambas as palavras não estão no vocabulário
ator - atuação: Uma ou ambas as palavras não estão no vocabulário
filme - protagonista: -0.11359719187021255


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))



Resultados com TF-IDF:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3


Resultados com Word Embeddings:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



In [30]:
import spacy

# Carregar modelo spaCy (pequeno)
try:
    nlp = spacy.load("pt_core_news_sm")  # Para português
    # Alternativa para inglês: nlp = spacy.load("en_core_web_sm")

    # Exemplo de texto
    texto = "O banco está cheio de dinheiro. Eu sentei no banco da praça."

    # Processar o texto
    doc = nlp(texto)

    # Examinar embeddings para cada ocorrência da palavra "banco"
    for token in doc:
        if token.text.lower() == "banco":
            contexto = doc[max(0, token.i-3):min(len(doc), token.i+4)]
            print(f"Contexto: {contexto}")
            print(f"Vetor (10 primeiras dimensões): {token.vector[:10]}")
            print(f"Dimensão do vetor: {len(token.vector)}")
            print("-" * 50)
except Exception as e:
    print(f"Erro ao carregar spaCy: {e}")
    print("Talvez seja necessário instalar os modelos com:")
    print("python -m spacy download pt_core_news_sm")
    print("ou")
    print("python -m spacy download en_core_web_sm")


Contexto: O banco está cheio de
Vetor (10 primeiras dimensões): [-2.761275    2.7341504   1.9384544   0.34338498  1.0899546  -4.300789
  1.5131965   3.7354984   1.1549678  -1.9115787 ]
Dimensão do vetor: 96
--------------------------------------------------
Contexto: Eu sentei no banco da praça.
Vetor (10 primeiras dimensões): [-2.8606548   3.2033174   4.2152634   2.3896182   4.4038973  -3.509808
  4.1706643   5.813535   -0.12308097 -0.5815916 ]
Dimensão do vetor: 96
--------------------------------------------------
